In [1]:
%load_ext autoreload
%autoreload 2

import plotly.io as pio
pio.renderers.default = 'iframe'

In [2]:
import pandas as pd
import plotly.express as px
import numpy as np
import pandas as pd
import re
from datetime import timedelta
import plotly.express as px

from op_analytics.datasources.defillama.dataaccess import DefiLlama
from op_analytics.coreutils.request import get_data, new_session

import urllib3
import warnings
pd.set_option('display.float_format', lambda x: '%.3f' % x)
urllib3.disable_warnings()
warnings.filterwarnings("ignore")

In [3]:
from op_analytics.coreutils.duckdb_inmem.localcopy import dump_local_copy, load_local_copy
from op_analytics.datasources.defillama.dataaccess import DefiLlama
from op_analytics.coreutils.duckdb_inmem.client import init_client


In [4]:
from dataclasses import dataclass

import polars as pl

from op_analytics.coreutils.bigquery.write import most_recent_dates
from op_analytics.coreutils.logger import structlog
from op_analytics.coreutils.partitioned.dailydatautils import dt_summary
from op_analytics.coreutils.request import get_data, new_session
from op_analytics.coreutils.threads import run_concurrently
from op_analytics.coreutils.time import now_dt, dt_fromisostr

from op_analytics.datasources.defillama.dataaccess import DefiLlama

log = structlog.get_logger()

YIELD_POOLS_ENDPOINT = "https://pro-api.llama.fi/slPjmq113xSROlwRStlysKOhP0coMlgQkcPd0lgLMV3sL316g8l8CQ/yields/pools"
YIELD_POOL_CHART_ENDPOINT = "https://pro-api.llama.fi/slPjmq113xSROlwRStlysKOhP0coMlgQkcPd0lgLMV3sL316g8l8CQ/yields/chart/{pool}"

YIELD_TABLE_LAST_N_DAYS = 7


@dataclass
class DefillamaYield:
    """Metadata and yield data for all pools.

    This is the result we obtain after fetching from the API and extracting the data
    that we need to ingest.
    """

    pool_yield_df: pl.DataFrame


def pull_yield_data(pull_pools: list[str] | None = None) -> pl.DataFrame:
    """
    Pulls and processes yield pool data from DeFiLlama.

    Args:
        pull_pools: list of pool IDs to process. Defaults to None (process all).

    Returns:
        A polars DataFrame containing joined pool and historical yield data.
    """
    session = new_session()

    # Get all pools data
    pools_data = get_data(session, YIELD_POOLS_ENDPOINT)
    pools_df = extract_pools_metadata(pools_data["data"])
    print(pools_df.schema)
    # Write pools metadata
    DefiLlama.YIELD_POOLS_METADATA.write(
        dataframe=pools_df.with_columns(dt=pl.lit(now_dt())),
        sort_by=["pool"],
    )

    # Get pool IDs to process
    pool_ids = pools_df["pool"].to_list() if pull_pools is None else pull_pools

    # Call the API endpoint for each pool in parallel
    urls = {pool: YIELD_POOL_CHART_ENDPOINT.format(pool=pool) for pool in pool_ids}
    historical_yield_data = run_concurrently(lambda x: get_data(session, x), urls, max_workers=4)

    # Extract historical yield data
    historical_yield_df = extract_historical_yield_data(historical_yield_data)

    # Merge historical data with pool metadata
    pool_yield_df = historical_yield_df.join(pools_df, on="pool", how="left")

    # # Write yield data
    DefiLlama.YIELD_POOLS_HISTORICAL.write(
        dataframe=most_recent_dates(
            pool_yield_df, n_dates=YIELD_TABLE_LAST_N_DAYS, date_column="dt"
        ),
        sort_by=["dt", "chain", "protocol_slug", "pool"],
    )

    return DefillamaYield(pool_yield_df=pool_yield_df)


def execute_pull():
    result = pull_yield_data()
    return {"pool_yield_df": dt_summary(result.pool_yield_df)}


def extract_pools_metadata(pools_data: list) -> pl.DataFrame:
    """Extract pools data and transform into dataframe"""
    records = [
        {
            "pool": pool["pool"],
            "protocol_slug": pool["project"],
            "chain": pool["chain"],
            "symbol": pool["symbol"],
            "underlying_tokens": pool.get("underlyingTokens", []),
            "reward_tokens": pool.get("rewardTokens", []),
            "il_risk": pool["ilRisk"],
            "is_stablecoin": pool["stablecoin"],
            "exposure": pool["exposure"],
            # "tvl_usd": pool["tvlUsd"], # TODO: Add this back in if we use TVL filtering
        }
        for pool in pools_data
    ]

    return pl.DataFrame(records)


def extract_historical_yield_data(data: dict) -> pl.DataFrame:
    """Extract historical yield data and transform into dataframe"""
    records = [
        {
            "pool": pool_id,
            "dt": dt_fromisostr(entry["timestamp"]),
            "tvl_usd": float(entry.get("tvlUsd", 0.0)),
            "apy": float(entry.get("apy", 0.0)),
            "apy_base": float(entry.get("apyBase") or 0.0),
            "apy_reward": float(entry.get("apyReward") or 0.0),
        }
        for pool_id, pool_data in data.items()
        for entry in pool_data["data"]
    ]

    return pl.DataFrame(records)


In [5]:
result = pull_yield_data()

2025-02-04 08:40:48 [info     ] Fetched from https://pro-api.llama.fi/slPjmq113xSROlwRStlysKOhP0coMlgQkcPd0lgLMV3sL316g8l8CQ/yields/pools: 4.42 seconds filename=request.py lineno=81 process=11115
Schema({'pool': String, 'protocol_slug': String, 'chain': String, 'symbol': String, 'underlying_tokens': List(String), 'reward_tokens': List(String), 'il_risk': String, 'is_stablecoin': Boolean, 'exposure': String})
2025-02-04 08:40:48 [info     ] done writing 21.5Krows 985.7KB filename=gcs_parquet.py lineno=57 maxrss=586039296 path=/Users/chuck/codebase/op-analytics/ozone/warehouse/defillama/yield_pools_metadata_v1/dt=2025-02-04/out.parquet process=11115 root=defillama/yield_pools_metadata_v1 rows=21538 size=985696
2025-02-04 08:40:48 [info     ] memory usage                   filename=gcs_parquet.py lineno=65 max_rss=586.04 process=11115 root=defillama/yield_pools_metadata_v1
2025-02-04 08:40:48 [debug    ] done writing defillama/yield_pools_metadata_v1 to LOCAL filename=writehelper.py linen

In [6]:
result.pool_yield_df.to_pandas()

,pool,dt,tvl_usd,apy,apy_base,apy_reward,protocol_slug,chain,symbol,underlying_tokens,reward_tokens,il_risk,is_stablecoin,exposure
0,747c1d2a-c668-4682-b9f9-296708a3dd90,2022-05-03,11074372760.000,3.600,3.600,0.000,lido,Ethereum,STETH,[0x0000000000000000000000000000000000000000],None,no,False,single
1,747c1d2a-c668-4682-b9f9-296708a3dd90,2022-05-04,11744709791.000,3.600,3.600,0.000,lido,Ethereum,STETH,[0x0000000000000000000000000000000000000000],None,no,False,single
2,747c1d2a-c668-4682-b9f9-296708a3dd90,2022-05-05,11008298772.000,3.600,3.600,0.000,lido,Ethereum,STETH,[0x0000000000000000000000000000000000000000],None,no,False,single
3,747c1d2a-c668-4682-b9f9-296708a3dd90,2022-05-06,10974211995.000,3.500,3.500,0.000,lido,Ethereum,STETH,[0x0000000000000000000000000000000000000000],None,no,False,single
4,747c1d2a-c668-4682-b9f9-296708a3dd90,2022-05-07,10736517627.000,3.500,3.500,0.000,lido,Ethereum,STETH,[0x0000000000000000000000000000000000000000],None,no,False,single
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14604,c1ca08e4-d618-415e-ad63-fcec58705469,2025-01-31,654794279.000,0.000,0.000,0.000,compound-v3,Ethereum,WBTC,[0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599],None,no,False,single
14605,c1ca08e4-d618-415e-ad63-fcec58705469,2025-02-01,642743900.000,0.000,0.000,0.000,compound-v3,Ethereum,WBTC,[0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599],None,no,False,single
14606,c1ca08e4-d618-415e-ad63-fcec58705469,2025-02-02,626745093.000,0.000,0.000,0.000,compound-v3,Ethereum,WBTC,[0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599],None,no,False,single
14607,c1ca08e4-d618-415e-ad63-fcec58705469,2025-02-03,656050255.000,0.000,0.000,0.000,compound-v3,Ethereum,WBTC,[0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599],None,no,False,single


In [7]:
from op_analytics.coreutils.duckdb_inmem.client import init_client

ctx = init_client()
client = ctx.client

tvl_view = DefiLlama.YIELD_POOLS_HISTORICAL.read(
    min_date="2021-01-01",
)

# Process protocol TVL
df_yield_pool = client.sql(f"""
    SELECT
        *
    FROM {tvl_view}
""").pl().to_pandas()

2025-02-04 08:40:51 [info     ] initialized duckdb at /var/folders/0l/4g9g4jcd4fx8f16h1f6xznw00000gn/T/7e0b2s84/op-analytics.duck.db filename=client.py lineno=94 process=11115
2025-02-04 08:40:51 [info     ] loaded vault from .env file    filename=vault.py lineno=32 process=11115
2025-02-04 08:40:51 [debug    ] loaded vault: 19 items         filename=vault.py lineno=79 process=11115
2025-02-04 08:40:51 [info     ] Reading data from 'defillama/yield_pools_historical_v1' with filters min_date=2021-01-01, max_date=None, date_range_spec=None filename=dailydata.py lineno=182 process=11115
2025-02-04 08:40:51 [info     ] querying markers for 'defillama/yield_pools_historical_v1' DateFilter(min_date=datetime.date(2021, 1, 1), max_date=None, datevals=None) filename=dailydata.py lineno=105 process=11115
2025-02-04 08:40:51 [debug    ] connecting to OPLABS Clickhouse client... filename=client.py lineno=36 process=11115
2025-02-04 08:40:52 [debug    ] initialized OPLABS Clickhouse client. filenam

RuntimeError: Query interrupted

In [ ]:
df_yield_pool[df_yield_pool.tvl_usd > 0]

In [21]:
borrow_df = result.pool_yield_df.to_pandas()

In [22]:
borrow_df["dt"] = pd.to_datetime(borrow_df["dt"])

In [165]:
borrow_df[
    (borrow_df.chain == "Ethereum")
    & (borrow_df.protocol_slug == "aave-v2")
    & (borrow_df.pool_meta == "main_pool")
    & (borrow_df.symbol == "STETH")
    # & (borrow_df.borrowable == True)
    # & (borrow_df.symbol.isin(["USDT", "USDC"])
    # & (borrow_df.dt == "2025-01-28")

]

,pool,dt,total_supply_usd,total_borrow_usd,debt_ceiling_usd,apy_base,apy_reward,apy_base_borrow,apy_reward_borrow,protocol_slug,...,symbol,underlying_tokens,reward_tokens,il_risk,is_stablecoin,exposure,borrowable,minted_coin,borrow_factor,pool_meta
47045,0e458a5b-fd99-4667-8706-dd938934cb0c,2022-02-28,0,0,0,0.000,0.000,0.000,0.000,aave-v2,...,STETH,[0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84],None,no,False,single,False,None,NaN,main_pool
47046,0e458a5b-fd99-4667-8706-dd938934cb0c,2022-03-01,0,0,0,0.000,0.000,0.000,0.000,aave-v2,...,STETH,[0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84],None,no,False,single,False,None,NaN,main_pool
47047,0e458a5b-fd99-4667-8706-dd938934cb0c,2022-03-02,0,0,0,0.000,0.000,0.000,0.000,aave-v2,...,STETH,[0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84],None,no,False,single,False,None,NaN,main_pool
47048,0e458a5b-fd99-4667-8706-dd938934cb0c,2022-03-03,0,0,0,0.000,0.000,0.000,0.000,aave-v2,...,STETH,[0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84],None,no,False,single,False,None,NaN,main_pool
47049,0e458a5b-fd99-4667-8706-dd938934cb0c,2022-03-04,0,0,0,0.000,0.000,0.000,0.000,aave-v2,...,STETH,[0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84],None,no,False,single,False,None,NaN,main_pool
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48105,0e458a5b-fd99-4667-8706-dd938934cb0c,2025-01-25,118837051,0,0,0.000,0.000,0.000,0.000,aave-v2,...,STETH,[0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84],None,no,False,single,False,None,NaN,main_pool
48106,0e458a5b-fd99-4667-8706-dd938934cb0c,2025-01-26,117298106,0,0,0.000,0.000,0.000,0.000,aave-v2,...,STETH,[0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84],None,no,False,single,False,None,NaN,main_pool
48107,0e458a5b-fd99-4667-8706-dd938934cb0c,2025-01-27,113142412,0,0,0.000,0.000,0.000,0.000,aave-v2,...,STETH,[0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84],None,no,False,single,False,None,NaN,main_pool
48108,0e458a5b-fd99-4667-8706-dd938934cb0c,2025-01-28,109879721,0,0,0.000,0.000,0.000,0.000,aave-v2,...,STETH,[0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84],None,no,False,single,False,None,NaN,main_pool


In [79]:
import plotly.graph_objects as go

# Filter the dataframe for WSTETH and the two protocols
filtered_wsteth_df = borrow_df[
    (borrow_df["chain"] == "Ethereum") &
    (borrow_df["symbol"].isin(["WSTETH", "STETH"])) &
    (borrow_df["protocol_slug"].isin(["aave-v3", "spark", "aave-v2"])) &
    (borrow_df["pool_meta"] == "main_pool") & 
    (borrow_df["dt"] >= "2022-09-21")
]

# Define custom colors
color_mapping = {
    "aave-v2": "black",
    "aave-v3": "purple",
    "spark": "magenta"
}

# Create the figure
fig = go.Figure()

for protocol in filtered_wsteth_df["protocol_slug"].unique():
    df_subset = filtered_wsteth_df[filtered_wsteth_df["protocol_slug"] == protocol]
    fig.add_trace(go.Scatter(
        x=df_subset["dt"],
        y=df_subset["total_supply_usd"],
        mode="lines",
        name=f"{protocol} - WSTETH",
        line=dict(color=color_mapping[protocol], width=3)  # Slightly thicker lines for visibility
    ))

# Update layout
fig.update_layout(
    title="WSTETH Total Supply on Aave v2, Aave v3, & Spark",
    xaxis_title="Date",
    yaxis_title="Total Supply (USD)",
    template="plotly_white",
    margin=dict(t=50, l=25, r=25, b=50),
    legend=dict(
        title="Protocol",
        yanchor="top",
        y=1,
        xanchor="left",
        x=1.02
    )
)

# Show plot
fig.show()


In [164]:
import plotly.express as px
import plotly.graph_objects as go

# Filter the dataframe
filtered_df = borrow_df[
    (borrow_df["chain"] == "Ethereum") &
    (borrow_df["is_stablecoin"] == True) &
    (borrow_df["protocol_slug"].isin(["aave-v3", "spark"])) &
    (borrow_df["pool_meta"] == "main_pool") &
    (borrow_df["borrowable"] == True) &
    (
        ((borrow_df["symbol"].isin(["USDC", "USDT", "GHO"])) & (borrow_df["protocol_slug"] == "aave-v3")) |
        ((borrow_df["symbol"] == "DAI") & (borrow_df["protocol_slug"] == "spark"))
    )
]

# Create a new column combining protocol and symbol
filtered_df["protocol_symbol"] = filtered_df["protocol_slug"] + " - " + filtered_df["symbol"]

# Define custom colors
color_mapping = {
    "aave-v3 - USDT": "green",
    "aave-v3 - USDC": "blue",
    "aave-v3 - GHO": "purple",
    "spark - DAI": "orange"
}

# Define line width (make DAI thicker)
line_width_mapping = {
    "aave-v3 - USDT": 2,
    "aave-v3 - USDC": 2,
    "aave-v3 - GHO": 2,
    "spark - DAI": 4  # Thicker line for DAI
}

# Create the figure
fig = go.Figure()

for protocol_symbol in ["aave-v3 - USDT", "aave-v3 - USDC", "aave-v3 - GHO", ""]:
    df_subset = filtered_df[filtered_df["protocol_symbol"] == protocol_symbol]
    fig.add_trace(go.Scatter(
        x=df_subset["dt"],
        y=df_subset["apy_base_borrow"],
        mode="lines",
        name=protocol_symbol,
        line=dict(color=color_mapping[protocol_symbol], width=line_width_mapping[protocol_symbol])
    ))

# Update layout
fig.update_layout(
    title="Borrow Rates Over Time",
    xaxis_title="Date",
    yaxis_title="Base Borrow APY",
    template="plotly_white",
    margin=dict(t=50, l=25, r=25, b=50),
    legend=dict(
        title="Protocol & Token",
        yanchor="top",
        y=1,
        xanchor="left",
        x=1.02
    )
)

# Show plot
fig.show()


In [152]:
import plotly.graph_objects as go

# Filter borrow APY data for Aave v3 and Spark (only selected tokens)
filtered_borrow_df = borrow_df[
    (borrow_df["chain"] == "Ethereum") &
    (borrow_df["is_stablecoin"] == True) &
    (borrow_df["protocol_slug"].isin(["aave-v3", "spark"])) &
    (borrow_df["pool_meta"] == "main_pool") &
    (borrow_df["borrowable"] == True) &
    (
        ((borrow_df["protocol_slug"] == "aave-v3") & (borrow_df["symbol"].isin(["USDT", "USDC", "GHO"]))) |
        ((borrow_df["protocol_slug"] == "spark") & (borrow_df["symbol"] == "DAI"))
    )
]

# Filter total supply data for WSTETH on Aave v3 and Spark
filtered_supply_df = borrow_df[
    (borrow_df["chain"] == "Ethereum") &
    (borrow_df["symbol"] == "WSTETH") &
    (borrow_df["protocol_slug"].isin(["aave-v3", "spark"])) &
    (borrow_df["pool_meta"] == "main_pool")
]

# Define colors
borrow_colors = {"USDT": "green", "USDC": "blue", "GHO": "purple", "DAI": "orange"}
supply_colors = {"aave-v3": "purple", "spark": "magenta"}

# Create figure
fig = go.Figure()

# Add total supply (Left Y-axis)
for protocol in filtered_supply_df["protocol_slug"].unique():
    df_subset = filtered_supply_df[filtered_supply_df["protocol_slug"] == protocol]
    fig.add_trace(go.Scatter(
        x=df_subset["dt"],
        y=df_subset["total_supply_usd"],
        mode="lines",
        name=f"{protocol} - WSTETH Supply",
        line=dict(color=supply_colors[protocol], width=3),  # Thicker lines for clarity
        yaxis="y1"
    ))

# Add borrow APY (Right Y-axis)
for symbol in filtered_borrow_df["symbol"].unique():
    df_subset = filtered_borrow_df[filtered_borrow_df["symbol"] == symbol]
    for protocol in df_subset["protocol_slug"].unique():
        df_proto = df_subset[df_subset["protocol_slug"] == protocol]
        line_width = 4 if (symbol == "DAI" and protocol == "spark") else 2  # DAI on Spark should be thicker
        fig.add_trace(go.Scatter(
            x=df_proto["dt"],
            y=df_proto["apy_base_borrow"],
            mode="lines",
            name=f"{protocol} - {symbol} Borrow APY",
            line=dict(color=borrow_colors[symbol], width=line_width),
            yaxis="y2"
        ))

# Update layout
fig.update_layout(
    title="Total Supply & Borrow APY for Aave v3 & Spark",
    xaxis_title="Date",
    yaxis=dict(
        title="Total Supply (USD)",
        side="left",
        showgrid=False
    ),
    yaxis2=dict(
        title="Borrow APY (%)",
        side="right",
        overlaying="y",
        showgrid=False
    ),
    template="plotly_white",
    margin=dict(t=50, l=25, r=50, b=50),
    legend=dict(
        title="Legend",
        yanchor="top",
        y=1.05,
        xanchor="left",
        x=1.02
    )
)

# Show plot
fig.show()


In [212]:
import plotly.graph_objects as go

# Filter for only Aave v3 and USDC
filtered_borrow_df = borrow_df[
    (borrow_df["chain"] == "Ethereum") &
    (borrow_df["is_stablecoin"] == True) &
    (borrow_df["protocol_slug"] == "aave-v3") &  # Only Aave v3
    (borrow_df["pool_meta"] == "main_pool") &
    (borrow_df["borrowable"] == True) &
    (borrow_df["symbol"] == "USDC")  # Only USDC
]

# Define colors
borrow_color = "blue"

# Create figure
fig = go.Figure()

# Add total borrow (Left Y-axis) - Solid Blue Line
fig.add_trace(go.Scatter(
    x=filtered_borrow_df["dt"],
    y=filtered_borrow_df["total_supply_usd"] - filtered_borrow_df["total_borrow_usd"],
    mode="lines",
    name="Aave v3 - USDC Total Borrow",
    line=dict(color=borrow_color, width=2),  # Solid blue
    yaxis="y1"
))

# Add borrow APY (Right Y-axis) - Dashed & Thicker Blue Line
fig.add_trace(go.Scatter(
    x=filtered_borrow_df["dt"],
    y=filtered_borrow_df["apy_base_borrow"],
    mode="lines",
    name="Aave v3 - USDC Borrow APY",
    line=dict(color="black", width=3, dash="dash"),  # Thicker dashed blue
    yaxis="y2"
))

# Update layout
fig.update_layout(
    title="Aave v3 USDC - APY increases as borrowable suppply decreases",
    xaxis_title="Date",
    yaxis=dict(
        title="Total USD Amount Borrowable (Supply - Borrowed)",
        side="left",
        showgrid=False
    ),
    yaxis2=dict(
        title="Borrow APY (%)",
        side="right",
        overlaying="y",
        showgrid=False
    ),
    template="plotly_white",
    margin=dict(t=50, l=25, r=50, b=50),
    legend=dict(
        title="Legend",
        yanchor="top",
        y=1.05,
        xanchor="left",
        x=1.02
    )
)

# Show plot
fig.show()


In [200]:
borrow_df[
    (borrow_df["chain"] == "Ethereum")
    & (borrow_df.protocol_slug == "spark")    
    & (borrow_df.dt == "2025-01-28")

]

,pool,dt,total_supply_usd,total_borrow_usd,debt_ceiling_usd,apy_base,apy_reward,apy_base_borrow,apy_reward_borrow,protocol_slug,...,symbol,underlying_tokens,reward_tokens,il_risk,is_stablecoin,exposure,borrowable,minted_coin,borrow_factor,pool_meta
5471,3b45941c-16cb-48c5-a490-16c6c4f1d86a,2025-01-28,2358738746,228446,0,0.000,0.000,0.000,0.000,spark,...,WSTETH,[0x7f39C581F595B53c5cb19bD0b3f8dA6c935E2Ca0],None,no,False,single,True,None,NaN,main_pool
7573,e26ce7d9-db75-4aa4-b1db-cc21ae17bdfb,2025-01-28,1150098662,1020902619,0,10.868,0.000,12.028,0.000,spark,...,DAI,[0x6B175474E89094C44Da98b954EedeAC495271d0F],None,no,True,single,True,None,NaN,main_pool
8205,24195b31-d749-445f-bf9e-b65aa025ebdd,2025-01-28,946752595,810529095,0,1.926,0.000,2.368,0.000,spark,...,WETH,[0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2],None,no,False,single,True,None,NaN,main_pool
28946,9da8a761-eb0f-4d41-ad05-0ebb2ed82913,2025-01-28,181951773,0,0,0.000,0.000,5.000,0.000,spark,...,WEETH,[0xCd5fE23C85820F7B72D0926FC9b05b43E359b7ee],None,no,False,single,False,None,NaN,main_pool
29067,00d1e1f8-b3c8-4350-a247-5da693d2d4dd,2025-01-28,181583139,1928336,0,0.001,0.000,0.071,0.000,spark,...,CBBTC,[0xcbB7C0000aB88B473b1f5aFd9ef808440eed33Bf],None,no,False,single,True,None,NaN,main_pool
33471,03406d3a-fcc4-4fe3-8809-7a95222951b6,2025-01-28,165860995,1686600,0,0.000,0.000,0.034,0.000,spark,...,WBTC,[0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599],None,no,False,single,False,None,NaN,main_pool
41572,d3694b72-5bc4-44c9-8ab6-1fc7941d216a,2025-01-28,122960482,0,0,0.000,0.000,0.000,0.000,spark,...,SUSDS,[0xa3931d71877C0E7a3148CB7Eb4463524FEc27fbD],None,no,True,single,False,None,NaN,main_pool
47043,8751078b-6be1-403b-ac44-9f11fe87d400,2025-01-28,103539214,26737,0,0.000,0.000,0.254,0.000,spark,...,RETH,[0xae78736Cd615f374D3085123A210448E74Fc6393],None,no,False,single,True,None,NaN,main_pool
270006,65ce8276-b4d9-41ba-9f6f-21fc374cf9bc,2025-01-28,1410430,1179583,0,8.257,0.000,10.381,0.000,spark,...,USDC,[0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48],None,no,True,single,True,None,NaN,main_pool
286047,8fbe28b8-140d-4e37-8804-5d2aba4daded,2025-01-28,1018609,780824,0,6.928,0.000,9.509,0.000,spark,...,USDT,[0xdAC17F958D2ee523a2206206994597C13D831ec7],None,no,True,single,True,None,NaN,main_pool


In [201]:
borrow_df.columns

Index(['pool', 'dt', 'total_supply_usd', 'total_borrow_usd',
       'debt_ceiling_usd', 'apy_base', 'apy_reward', 'apy_base_borrow',
       'apy_reward_borrow', 'protocol_slug', 'chain', 'symbol',
       'underlying_tokens', 'reward_tokens', 'il_risk', 'is_stablecoin',
       'exposure', 'borrowable', 'minted_coin', 'borrow_factor', 'pool_meta'],
      dtype='object')

In [101]:
steth_df = df_yield_pool[
    (df_yield_pool.chain == "Ethereum")
    & (df_yield_pool.symbol == "STETH") 
    & (df_yield_pool.protocol_slug == "lido")
][["dt", "apy"]].rename(columns={"apy": "steth_native_apy"})

In [109]:
supply_df = borrow_df[
    (borrow_df["chain"] == "Ethereum")
    & (borrow_df["symbol"] == "WSTETH")
    & (borrow_df["protocol_slug"].isin(["aave-v3", "spark"]))
    & (borrow_df["pool_meta"] == "main_pool")
][["dt", "total_supply_usd", "total_borrow_usd", "apy_base", "protocol_slug", "symbol"]].rename(
    columns={
        "apy_base": "supply_apy",
        "symbol": "supply_token",
    }
)

In [138]:
stable_borrow_df = (
    borrow_df[
        (borrow_df["chain"] == "Ethereum")
        & (borrow_df["is_stablecoin"] == True)
        & (borrow_df["protocol_slug"].isin(["aave-v3", "spark"]))
        & (borrow_df["pool_meta"] == "main_pool")
        & (borrow_df["borrowable"] == True)
        & (
            (
                (borrow_df["protocol_slug"] == "aave-v3")
                & (borrow_df["symbol"].isin(["USDT", "USDC", "GHO"]))
            )
            | ((borrow_df["protocol_slug"] == "spark") & (borrow_df["symbol"] == "DAI"))
        )
    ][["dt", "apy_base_borrow", "protocol_slug", "symbol"]].rename(
        columns={"symbol": "borrow_token", "apy_base_borrow": "borrow_apy"}
    )
)

In [139]:
susde_df = df_yield_pool[
    (df_yield_pool.chain == "Ethereum")
    & (df_yield_pool.symbol == "SUSDE") 
    & (df_yield_pool.protocol_slug == "ethena-usde")
][["dt", "apy"]].rename(columns={"apy": "susde_native_apy"})

In [140]:
strategy_df = (
    pd.merge(steth_df, supply_df, on="dt", how="left")
    .merge(stable_borrow_df, on=["dt", "protocol_slug"], how="left")
    .merge(susde_df, on="dt", how="left")
    
)

In [141]:
strategy_df["borrow_net_apy"] = (
    strategy_df.steth_native_apy
    + strategy_df.supply_apy
    - strategy_df.borrow_apy    
)

In [142]:
strategy_df["trade_yield"] = (
    strategy_df.steth_native_apy
    + strategy_df.supply_apy
    + strategy_df.susde_native_apy
    - strategy_df.borrow_apy

    
)

In [143]:
strategy_df

,dt,steth_native_apy,total_supply_usd,total_borrow_usd,supply_apy,protocol_slug,supply_token,borrow_apy,borrow_token,susde_native_apy,borrow_net_apy,trade_yield
0,2022-05-03,3.600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-05-04,3.600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-05-05,3.600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-05-06,3.500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-05-07,3.500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2473,2025-01-27,3.075,2448236242.000,236104.000,0.000,spark,WSTETH,12.028,DAI,14.780,-8.953,5.827
2474,2025-01-28,2.906,3870713354.000,1144511678.000,0.182,aave-v3,WSTETH,9.699,USDT,14.729,-6.611,8.119
2475,2025-01-28,2.906,3870713354.000,1144511678.000,0.182,aave-v3,WSTETH,9.681,USDC,14.729,-6.593,8.136
2476,2025-01-28,2.906,3870713354.000,1144511678.000,0.182,aave-v3,WSTETH,9.000,GHO,14.729,-5.912,8.817


In [215]:
import plotly.graph_objects as go

# Define color mapping for the borrow tokens
color_mapping = {
    "DAI": "orange",
    "USDT": "green",
    "USDC": "blue",
    "GHO": "purple"
}

# Filter data for dates starting from Feb 16, 2024
strategy_df_filtered = strategy_df[strategy_df["dt"] >= "2024-02-16"]

# Create the figure
fig = go.Figure()

# Iterate through unique protocol_slug + borrow_token combinations
for (protocol, token) in strategy_df_filtered[["protocol_slug", "borrow_token"]].drop_duplicates().sort_values(by="protocol_slug", ascending=True).itertuples(index=False):
    df_subset = strategy_df_filtered[
        (strategy_df_filtered["protocol_slug"] == protocol) &
        (strategy_df_filtered["borrow_token"] == token)
    ]
    
    fig.add_trace(go.Scatter(
        x=df_subset["dt"],
        y=df_subset["borrow_net_apy"],
        mode="lines",
        name=f"{protocol} - {token}",
        line=dict(
            color=color_mapping.get(token, "gray"),
            width=3 if token == "DAI" else 2 
        )
    ))

# Update layout
fig.update_layout(
    title="Net Cost to Borrow Stables against wstETH",
    xaxis_title="Date",
    yaxis_title="Net Cost to Borrow",
    template="plotly_white",
    margin=dict(t=50, l=25, r=25, b=50),
    legend=dict(
        title="Protocol - Borrow Token",
        yanchor="top",
        y=1.02,
        xanchor="left",
        x=1.02
    )
)

# Show plot
fig.show()


In [214]:
import plotly.graph_objects as go

# Define color mapping for the borrow tokens
color_mapping = {
    "USDT": "green",
    "USDC": "blue",
    "DAI": "orange"
}

# Define line width mapping (make DAI thicker)
line_width_mapping = {
    "USDT": 2,
    "USDC": 2,
    "DAI": 4  # Thicker line for DAI
}

# Filter data for dates starting from Feb 16, 2024
strategy_df_filtered = strategy_df[
    (strategy_df["dt"] >= "2024-02-16")
    & (strategy_df.borrow_token != "GHO")
]

# Create the figure
fig = go.Figure()

# Iterate through unique protocol_slug + borrow_token combinations
for (protocol, token) in strategy_df_filtered[["protocol_slug", "borrow_token"]].drop_duplicates().itertuples(index=False):
    df_subset = strategy_df_filtered[
        (strategy_df_filtered["protocol_slug"] == protocol) &
        (strategy_df_filtered["borrow_token"] == token)
    ]
    
    fig.add_trace(go.Scatter(
        x=df_subset["dt"],
        y=df_subset["trade_yield"],
        mode="lines",
        name=f"{protocol} - {token}",
        line=dict(
            color=color_mapping.get(token, "gray"),  # Default to gray for unknown tokens
            width=line_width_mapping.get(token, 2)   # Default width is 2 unless specified
        )
    ))

# Update layout
fig.update_layout(
    title="Cost to Supply wstETH, Borrow Stables, and Swap to sUSDe",
    xaxis_title="Date",
    yaxis_title="Trade Yield",
    template="plotly_white",
    margin=dict(t=50, l=25, r=25, b=50),
    legend=dict(
        title="Protocol - Borrow Token",
        yanchor="top",
        y=1.02,
        xanchor="left",
        x=1.02
    )
)

# Show plot
fig.show()


## Pendle

In [8]:
from op_analytics.coreutils.duckdb_inmem.client import init_client

ctx = init_client()
client = ctx.client

tvl_view = DefiLlama.PROTOCOL_TOKEN_TVL_BREAKDOWN.read(
    min_date="2024-01-01",
)

# Process protocol TVL
df_tvl_breakdown = client.sql(f"""
    SELECT
        *
    FROM {tvl_view}
""").pl().to_pandas()

2025-02-04 08:42:34 [info     ] Reading data from 'defillama/protocol_token_tvl_breakdown_v1' with filters min_date=2024-01-01, max_date=None, date_range_spec=None filename=dailydata.py lineno=182 process=11115
2025-02-04 08:42:34 [info     ] querying markers for 'defillama/protocol_token_tvl_breakdown_v1' DateFilter(min_date=datetime.date(2024, 1, 1), max_date=None, datevals=None) filename=dailydata.py lineno=105 process=11115
2025-02-04 08:42:35 [info     ] 1145 markers found             filename=dailydata.py lineno=119 max_dt=2025-02-03 min_dt=2024-01-01 process=11115
2025-02-04 08:42:35 [info     ] 400 distinct paths             filename=dailydata.py lineno=129 process=11115
2025-02-04 08:47:25 [info     ] registered view: 'defillama_protocol_token_tvl_breakdown_v1' using 400 parquet paths filename=client.py lineno=291 process=11115
┌───────────────────────────────────────────┐
│                   name                    │
│                  varchar                  │
├────────────

In [9]:
df_tvl_breakdown

,protocol_slug,chain,token,app_token_tvl,app_token_tvl_usd,protocol_name,protocol_category,parent_protocol,misrepresented_tokens,is_protocol_misrepresented,is_double_counted,to_filter_out,dt
0,acala-dollar,Acala,ACA,146394293.350,14709405.807,Acala Dollar,CDP,acala-dollar,1,0,0,0,2024-01-01
1,acala-dollar,Acala,DOT,18165.438,148956.595,Acala Dollar,CDP,acala-dollar,1,0,0,0,2024-01-01
2,acala-dollar,Acala,USDT,6330450.659,6330450.659,Acala Dollar,CDP,acala-dollar,1,0,0,0,2024-01-01
3,acala-euphrates,Acala,DOT,1110062.013,9102508.503,Acala Euphrates,Yield,acala-euphrates,1,0,1,0,2024-01-01
4,acala-euphrates,Acala,LDOT,13013285.527,14705012.646,Acala Euphrates,Yield,acala-euphrates,1,0,1,0,2024-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
31637586,zkswap-era,zkSync Era-staking,WETH,0.055,155.380,zkSwap Era,Dexs,zkswap-era,1,0,0,1,2025-02-03
31637587,zkswap-v2,zkSync Era-staking,ZF,180440435.018,914780.678,zkSwap V2,Dexs,zkswap-finance,0,0,0,1,2025-02-03
31637588,sablier-lockup,zkSync Era-vesting,CHEEMS,250020.000,0.202,Sablier Lockup,Payments,sablier-finance,0,0,0,1,2025-02-03
31637589,sablier-lockup,zkSync Era-vesting,ZFI,265337201.401,5080337.101,Sablier Lockup,Payments,sablier-finance,0,0,0,1,2025-02-03


In [10]:
import plotly.graph_objects as go

# Filter DataFrame for Base chain and Pendle protocol
df_filtered = df_tvl_breakdown[
    (df_tvl_breakdown["chain"] == "Base") &
    (df_tvl_breakdown["parent_protocol"] == "pendle")
]

# Create figure
fig = go.Figure()

# Loop through unique tokens to create separate lines
for token in df_filtered["token"].unique():
    df_subset = df_filtered[df_filtered["token"] == token]
    fig.add_trace(go.Scatter(
        x=df_subset["dt"],
        y=df_subset["app_token_tvl_usd"],
        mode="lines",
        name=token,
        line=dict(width=3)
    ))

# Update layout
fig.update_layout(
    title="Pendle Token TVL Breakdown on Base",
    xaxis_title="Date",
    yaxis_title="TVL (USD)",
    template="plotly_white",
    margin=dict(t=50, l=25, r=25, b=50),
    legend=dict(
        title="Token",
        yanchor="top",
        y=1,
        xanchor="left",
        x=1.02
    )
)

# Show plot
fig.show()


In [27]:
borrow_df[
    (borrow_df.symbol == "WSTETH")
    & (borrow_df.chain == "Ethereum")
    & (borrow_df.dt >= "2023-01-01")
].protocol_slug.unique()

array(['aave-v3', 'spark', 'makerdao', 'compound-v3', 'morpho-blue',
       'fluid-lending', 'crvusd', 'strike', 'fraxlend', 'silo-v1',
       'inverse-finance-firm', 'dforce', 'prismalst', 'gravita-protocol'],
      dtype=object)

In [25]:
borrow_df[
    (borrow_df.symbol == "WSTETH")
    & (borrow_df.chain == "Ethereum")
    & (borrow_df.protocol_slug == "spark")
    & (borrow_df.dt >= "2023-01-01")
    & (borrow_df.pool == "d541708e-1283-4feb-bc7a-457fc5f8db2c")
]

,pool,dt,total_supply_usd,total_borrow_usd,debt_ceiling_usd,apy_base,apy_reward,apy_base_borrow,apy_reward_borrow,protocol_slug,...,symbol,underlying_tokens,reward_tokens,il_risk,is_stablecoin,exposure,borrowable,minted_coin,borrow_factor,pool_meta
12064,d541708e-1283-4feb-bc7a-457fc5f8db2c,2024-11-08,624674139,301385532,0,0.622,0.000,1.357,0.000,aave-v3,...,WSTETH,[0x7f39C581F595B53c5cb19bD0b3f8dA6c935E2Ca0],None,no,False,single,True,None,NaN,lido-market
12065,d541708e-1283-4feb-bc7a-457fc5f8db2c,2024-11-09,657591500,315944660,0,0.617,0.000,1.351,0.000,aave-v3,...,WSTETH,[0x7f39C581F595B53c5cb19bD0b3f8dA6c935E2Ca0],None,no,False,single,True,None,NaN,lido-market
12066,d541708e-1283-4feb-bc7a-457fc5f8db2c,2024-11-10,659516954,323299374,0,0.642,0.000,1.379,0.000,aave-v3,...,WSTETH,[0x7f39C581F595B53c5cb19bD0b3f8dA6c935E2Ca0],None,no,False,single,True,None,NaN,lido-market
12067,d541708e-1283-4feb-bc7a-457fc5f8db2c,2024-11-11,714208367,342235671,0,0.614,0.000,1.348,0.000,aave-v3,...,WSTETH,[0x7f39C581F595B53c5cb19bD0b3f8dA6c935E2Ca0],None,no,False,single,True,None,NaN,lido-market
12068,d541708e-1283-4feb-bc7a-457fc5f8db2c,2024-11-12,668385136,334217528,0,0.668,0.000,1.406,0.000,aave-v3,...,WSTETH,[0x7f39C581F595B53c5cb19bD0b3f8dA6c935E2Ca0],None,no,False,single,True,None,NaN,lido-market
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12142,d541708e-1283-4feb-bc7a-457fc5f8db2c,2025-01-25,778052218,313612097,0,0.300,0.000,0.784,0.000,aave-v3,...,WSTETH,[0x7f39C581F595B53c5cb19bD0b3f8dA6c935E2Ca0],None,no,False,single,True,None,NaN,lido-market
12143,d541708e-1283-4feb-bc7a-457fc5f8db2c,2025-01-26,772177209,311031751,0,0.300,0.000,0.783,0.000,aave-v3,...,WSTETH,[0x7f39C581F595B53c5cb19bD0b3f8dA6c935E2Ca0],None,no,False,single,True,None,NaN,lido-market
12144,d541708e-1283-4feb-bc7a-457fc5f8db2c,2025-01-27,774567379,298747764,0,0.275,0.000,0.750,0.000,aave-v3,...,WSTETH,[0x7f39C581F595B53c5cb19bD0b3f8dA6c935E2Ca0],None,no,False,single,True,None,NaN,lido-market
12145,d541708e-1283-4feb-bc7a-457fc5f8db2c,2025-01-28,750569027,289064012,0,0.274,0.000,0.749,0.000,aave-v3,...,WSTETH,[0x7f39C581F595B53c5cb19bD0b3f8dA6c935E2Ca0],None,no,False,single,True,None,NaN,lido-market


In [27]:
session = new_session()

# Get all pools data
pools_data = get_data(session, YIELD_POOLS_ENDPOINT)
pools_df = extract_pools_data(pools_data["data"]).to_pandas()

2025-01-29 10:07:56 [info     ] Fetched from https://pro-api.llama.fi/slPjmq113xSROlwRStlysKOhP0coMlgQkcPd0lgLMV3sL316g8l8CQ/yields/poolsBorrow: 0.20 seconds filename=request.py lineno=81 process=68721


In [28]:
pools_df

,pool,protocol_slug,chain,symbol,underlying_tokens,reward_tokens,il_risk,is_stablecoin,exposure,ltv,borrowable,minted_coin,borrow_factor
0,e880e828-ca59-4ec6-8d4f-27182a4dc23d,aave-v3,Ethereum,WETH,[0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2],None,no,False,single,0.805,True,None,NaN
1,db678df9-3281-4bc2-a8bb-01160ffd6d48,aave-v3,Ethereum,WEETH,[0xCd5fE23C85820F7B72D0926FC9b05b43E359b7ee],None,no,False,single,0.775,True,None,NaN
2,e6435aae-cbe9-4d26-ab2c-a4d533db9972,aave-v3,Ethereum,WSTETH,[0x7f39C581F595B53c5cb19bD0b3f8dA6c935E2Ca0],None,no,False,single,0.785,True,None,NaN
3,7e382157-b1bc-406d-b17b-facba43b716e,aave-v3,Ethereum,WBTC,[0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599],None,no,False,single,0.730,True,None,NaN
4,f981a304-bb6c-45b8-b0c5-fd2f515ad23a,aave-v3,Ethereum,USDT,[0xdAC17F958D2ee523a2206206994597C13D831ec7],None,no,True,single,0.750,True,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1644,1806b085-e7d0-4ec9-b293-29c150c7fcf8,silo-v1,Ethereum,USDT,"[0xdAC17F958D2ee523a2206206994597C13D831ec7, 0...",None,no,True,single,0.900,None,None,NaN
1645,1e00ac2b-0c3c-4b1f-95be-9378f98d2b40,aave-v3,Ethereum,GHO,[0x40D16FC0246aD3160Ccc09B8D0D3A2cD28aE6C2f],None,no,True,single,NaN,True,GHO,NaN
1646,2439cc3f-5984-4e63-a949-8366c35c57a2,silo-v1,Ethereum,PT-WEETH-27JUN2024,"[0xc69Ad9baB1dEE23F4605a82b3354F8E40d1E5966, 0...",None,yes,False,multi,0.850,None,None,NaN
1647,3c34c22f-79eb-418a-b231-f31b0caf8d61,silo-v1,Ethereum,STKCVXSTETH-NG-F-SILO,"[0x470f9f350642eDF3CB3f2a6eB4569c0219386F0b, 0...",None,yes,False,multi,0.850,None,None,NaN


In [11]:
borrow = pools_data["data"][0]

In [ ]:
/yields/pools

In [ ]:
e880e828-ca59-4ec6-8d4f-27182a4dc23d

In [14]:
borrow

{'chain': 'Ethereum',
 'project': 'aave-v3',
 'symbol': 'WETH',
 'tvlUsd': 712932922,
 'apyBase': 1.96095,
 'apyReward': None,
 'apy': 1.96095,
 'rewardTokens': None,
 'pool': 'e880e828-ca59-4ec6-8d4f-27182a4dc23d',
 'apyPct1D': 0.05369,
 'apyPct7D': 0.06972,
 'apyPct30D': 0.09553,
 'stablecoin': False,
 'ilRisk': 'no',
 'exposure': 'single',
 'predictions': {'predictedClass': 'Stable/Up',
  'predictedProbability': 94,
  'binnedConfidence': 3},
 'poolMeta': None,
 'mu': 1.87273,
 'sigma': 0.01931,
 'count': 721,
 'outlier': False,
 'underlyingTokens': ['0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2'],
 'il7d': None,
 'apyBase7d': None,
 'apyMean30d': 1.84372,
 'volumeUsd1d': None,
 'volumeUsd7d': None,
 'apyBaseInception': None,
 'apyBaseBorrow': 2.63078,
 'apyRewardBorrow': None,
 'totalSupplyUsd': 5792539003,
 'totalBorrowUsd': 5079606082,
 'debtCeilingUsd': None,
 'ltv': 0.805,
 'borrowable': True,
 'mintedCoin': None,
 'borrowFactor': None}

In [16]:
5079606082/(5792539003+712932922)

0.7808205370127702

In [24]:
chain_yield_df = pools_df.groupby("chain").agg({"tvl_usd": "sum", "pool": "nunique"}).reset_index()

In [32]:
chain_yield_df[chain_yield_df.chain.str.contains("zk")]

,chain,tvl_usd,pool
28,Cronos_zkevm,2938777,6
74,Polygon zkEVM,5193751,47
97,zkSync Era,99682496,77


In [48]:
chain_yield_df[chain_yield_df.tvl_usd > 50_000_000].chain.to_list()

['Algorand',
 'Aptos',
 'Arbitrum',
 'Avalanche',
 'BSC',
 'Base',
 'Bitcoin',
 'Cardano',
 'Core',
 'Cronos',
 'Ethereum',
 'Filecoin',
 'Flare',
 'Fraxtal',
 'Gnosis',
 'Hedera',
 'Kava',
 'Linea',
 'Mode',
 'MultiversX',
 'Optimism',
 'Osmosis',
 'Polygon',
 'Scroll',
 'Solana',
 'Sonic',
 'Starknet',
 'Sui',
 'Ton',
 'Tron',
 'zkSync Era']

In [57]:
chain_list = chain_yield_df[chain_yield_df.tvl_usd > 50_000_000].chain.to_list()

pools_df[
    (pools_df.chain.isin(chain_list))
    & (pools_df.tvl_usd < 50_000)
].sample(20)

,pool,protocol_slug,chain,symbol,underlying_tokens,tvl_usd
18245,e4de64db-c623-440a-8b4a-7959164c0705,kiloex,BSC,SOLVBTC,[0x4aae823a6a0b376de6a78e74ecc5b079d38cbcf7],13213
14740,e148ba96-42b4-4a64-9091-bc3d269c314a,uniswap-v2,Base,TRUMP-WETH,"[0x06342c8681498b49cb548c1b6e9213454290e707, 0...",27141
12417,c7f10b6c-d289-40aa-8db2-79d5c2505327,raydium,Solana,WSOL-RF,None,49180
15108,8a0be992-6026-4b74-808a-ac98dc44fdde,uniswap-v3,Arbitrum,UXLINK-WETH,"[0x1a6b3a62391eccaaa992ade44cd4afe6bec8cff1, 0...",24506
17517,695f07e6-0a68-45f7-a876-ac214647fbf2,cetus-amm,Sui,PEWS-SUI,[0xace82dd859217d7f3b0cd82ac180449b8de25c4df55...,14966
19680,e55861ed-4580-4950-a9f0-1618b93c77c1,raydium,Solana,WSOL-FRANCE,None,10497
13846,24ecd148-4a08-4196-afde-fe22f07c5a26,cetus-amm,Sui,USDC-FOMO,[0xdba34672e30cb065b1f93e3ab55318768fd6fef66c1...,33917
13386,0bea8700-111c-43fc-a564-2c9bd9ebcd78,joe-v2.1,Arbitrum,VEIN-WETH,"[0xc4a70668dec1da9862d9d20bf67d1ef4ee182450, 0...",37711
17114,e1a7c8ca-0445-4511-ab84-86139fb4054c,raydium,Solana,WSOL-PSYCHO,None,16111
19172,e7ff2754-dc17-4dba-801a-ad4f2ab7443e,joe-dex,Avalanche,MIM-WINE,"[0x130966628846bfd36ff31a822705796e8cb8c18d, 0...",11263


In [47]:
chain_yield_df[chain_yield_df.chain.str.contains("Ink")]

,chain,tvl_usd,pool


In [7]:
ctx = init_client()
client = ctx.client

tvl_view = DefiLlama.PROTOCOL_TOKEN_TVL_BREAKDOWN.read(min_date="2024-08-01")

df_protocol_tvl = client.sql(f"""
    SELECT
        *
    FROM {tvl_view}
""").pl().to_pandas()

2025-01-24 12:32:29 [info     ] Reading data from 'defillama/protocol_token_tvl_breakdown_v1' with filters min_date=2024-08-01, max_date=None, date_range_spec=None filename=dailydata.py lineno=187 process=66595
2025-01-24 12:32:29 [info     ] querying markers for 'defillama/protocol_token_tvl_breakdown_v1' DateFilter(min_date=datetime.date(2024, 8, 1), max_date=None, datevals=None) filename=dailydata.py lineno=105 process=66595
2025-01-24 12:32:29 [info     ] 391 markers found              filename=dailydata.py lineno=119 max_dt=2025-01-23 min_dt=2024-08-01 process=66595
2025-01-24 12:32:29 [info     ] 176 distinct paths             filename=dailydata.py lineno=129 process=66595
2025-01-24 12:32:31 [info     ] registered view: 'defillama_protocol_token_tvl_breakdown_v1' using 176 parquet paths filename=client.py lineno=291 process=66595
┌───────────────────────────────────────────┐
│                   name                    │
│                  varchar                  │
├────────────

In [8]:

YIELD_ENDPOINT = "https://yields.llama.fi/pools"
session = new_session()
yield_data = get_data(session, YIELD_ENDPOINT)

yield_lists = []

for yield_pool in yield_data["data"]:
    chain = yield_pool["chain"]
    project = yield_pool["project"]
    pool = yield_pool["pool"]
    symbol = yield_pool["symbol"]
    underlying_tokens = yield_pool["underlyingTokens"] 
    yield_lists.append(
        {
            "chain_name": chain,
            "protocol_slug": project,
            "pool": pool,
            "symbol": symbol,
            "underlying_tokens": underlying_tokens
        }
    )

df_yield = pd.DataFrame(yield_lists)


2025-01-24 12:32:51 [info     ] Fetched from https://yields.llama.fi/pools: 0.82 seconds filename=request.py lineno=81 process=66595


In [9]:
ethena_pool = df_yield[df_yield.protocol_slug == "ethena-usde"].pool.to_list()
aave_pools = df_yield[
    (df_yield.protocol_slug == "aave-v3")
    & (df_yield.chain_name == "Ethereum")
].pool.to_list()

yield_pools = ethena_pool + aave_pools


In [10]:
YIELD_POOL_ENDPOINT = "https://yields.llama.fi/chart/{pool}"
session = new_session()
yield_data = get_data(session, YIELD_ENDPOINT)

pool_dfs = []
for pool in yield_pools:
    url = YIELD_POOL_ENDPOINT.format(pool=pool)
    pool_chart = get_data(session, url)
    df_pool = pd.DataFrame(pool_chart["data"])
    df_pool["pool"] = pool
    
    pool_dfs.append(df_pool)



2025-01-24 12:33:07 [info     ] Fetched from https://yields.llama.fi/pools: 0.62 seconds filename=request.py lineno=81 process=66595
2025-01-24 12:33:08 [info     ] Fetched from https://yields.llama.fi/chart/66985a81-9c51-46ca-9977-42b4fe7bc6df: 0.64 seconds filename=request.py lineno=81 process=66595
2025-01-24 12:33:08 [info     ] Fetched from https://yields.llama.fi/chart/db678df9-3281-4bc2-a8bb-01160ffd6d48: 0.63 seconds filename=request.py lineno=81 process=66595
2025-01-24 12:33:09 [info     ] Fetched from https://yields.llama.fi/chart/7e382157-b1bc-406d-b17b-facba43b716e: 0.74 seconds filename=request.py lineno=81 process=66595
2025-01-24 12:33:10 [info     ] Fetched from https://yields.llama.fi/chart/e6435aae-cbe9-4d26-ab2c-a4d533db9972: 0.76 seconds filename=request.py lineno=81 process=66595
2025-01-24 12:33:10 [info     ] Fetched from https://yields.llama.fi/chart/d3be212c-f337-40e2-87fc-fb636a3de38f: 0.44 seconds filename=request.py lineno=81 process=66595
2025-01-24 12:33:

In [11]:
df_pools = pd.concat(pool_dfs)

In [12]:
df_pools = pd.merge(
    df_pools,
    df_yield,
    on="pool",
    how="left"
)

In [13]:
pool_counts = df_pools.groupby(["pool", "symbol"]).agg({"timestamp": "nunique"}).reset_index().sort_values(by="symbol")

In [14]:
symbols = [
    'CBBTC',
    'CBETH',
    'CRVUSD',
    'ETHX',
    'FRAX',
    'LUSD',
    'OSETH',
    'PYUSD',
    'RETH',
    'SUSDE',
    'TBTC',
    'USDC',
    'USDE',
    'USDS',
    'USDT',
    'WBTC',
    'WEETH',
    'WETH',
    'WSTETH'
]

In [15]:
pools_to_use = (
    pool_counts[
    (pool_counts.timestamp > 40) # manual filter, need to understand what's happening 
    & pool_counts.symbol.isin(symbols)
    & (pool_counts.pool != "29932dea-cd71-44c3-95bd-3e1525f4e3dd")
    ].pool.to_list()
)

In [16]:
df_pools["slug_symbol"] = df_pools["symbol"] + "-" + df_pools["protocol_slug"]

In [17]:
df_pools = df_pools.sort_values(by=["pool", "timestamp"])
df_pools["timestamp"] = pd.to_datetime(df_pools["timestamp"], utc=True)

df_pools["apy_7d_rolling_avg"] = df_pools.groupby("pool").apply(
    lambda group: group.sort_values("timestamp").rolling("7d", on="timestamp")["apy"].mean()
).reset_index(level=0, drop=True)

In [18]:
df_pools["timestamp"] = pd.to_datetime(df_pools["timestamp"])

# Create the line plot
fig = px.line(
    df_pools[
       df_pools.pool.isin(pools_to_use)
        & (df_pools.timestamp >= "2024-09-01")
        & (df_pools.symbol.isin(["USDE", "SUSDE", "USDT", "USDC"]))
    ]
    ,
    x="timestamp",
    y="apy_7d_rolling_avg",
    color="symbol",
    title="Yield pool APY over time",
    labels={"dt": "Date", "app_token_tvl_usd": "TVL (USD)", "symbol": "Token"},
)

# Customize layout for better visualization
fig.update_layout(
    template="plotly_white",
    xaxis_title="Date",
    yaxis_title="TVL (USD)",
    legend_title="Token",
    margin=dict(t=50, l=25, r=25, b=50),
)

# Show plot
fig.show()

In [25]:
# drop duplicates due to an ongoing data upload issue
df_all = pd.merge(
    df_metadata.drop_duplicates(), 
    df_protocol_tvl.drop_duplicates(), 
    on="protocol_slug",
    how="left"
)


In [26]:
# Merge data and join alignment and token categories
df_all = pd.merge(df_all, alignment_df, on="chain", how="left")
df_all["alignment"] = df_all["alignment"].fillna("Other")
df_all = pd.merge(df_all, token_categories, on="token", how="left")
df_all["token_category"] = df_all["token_category"].fillna("Other")


In [27]:
# Chain level misrepresented tokens
df_misrep = (
    df_all[df_all.dt == df_all["dt"].max()-pd.Timedelta(days=1)]
    [["protocol_slug", "chain", "misrepresented_tokens", "token"]]
    .groupby(["protocol_slug", "chain", "misrepresented_tokens"])
    .agg(
        token_count=("token", "nunique"),
        has_usdt=("token", lambda x: 1 if "USDT" in x.values else 0)
    )
    .reset_index()
)

df_misrep["chain_misrepresented_tokens"] = (
    (df_misrep["misrepresented_tokens"] == 1) 
    & (df_misrep["token_count"] == 1) 
    & (df_misrep["has_usdt"] == 1)
).astype(int)

df_all = pd.merge(
    df_all, 
    df_misrep[["protocol_slug", "chain", "chain_misrepresented_tokens"]], 
    on=["protocol_slug", "chain"],
    how="left"
)

In [28]:
protocol_slug = "aave-v3"

In [29]:
df_borrow = df_all[
    (df_all.dt >= "2023-12-15")
    # & (df_all.chain.str.contains("borrow"))
    & (df_all.chain == "Ethereum-borrowed")
    & (df_all.protocol_slug == protocol_slug)
    # & (df_all.token_category == "Stablecoins")
    # & (df_all.token.isin(["SUSDE", "USDT", "USDC", "DAI"]))
].groupby(["dt", "protocol_name", "token"]).agg({"app_token_tvl_usd": "sum"}).reset_index()

In [30]:
df_borrow["token"] = df_borrow["token"] + "_borrowed"

In [31]:
df_lending = df_all[
    (df_all.dt >= "2023-12-15")
    # & (df_all.chain.str.contains("borrow"))
    & (df_all.chain == "Ethereum")
    & (df_all.protocol_slug == protocol_slug)
    # & (df_all.token_category == "Stablecoins")
    & (df_all.token.isin(["SUSDE", "USDT", "USDC"]))

].groupby(["dt", "protocol_name", "token"]).agg({"app_token_tvl_usd": "sum"}).reset_index()

In [32]:
df_lending_borrow = pd.concat([df_lending, df_borrow])

In [33]:
df_lending_borrow["dt"] = pd.to_datetime(df_lending_borrow["dt"])

# Create the line plot
fig = px.line(
    df_lending_borrow[
        (df_lending_borrow.token.isin(["SUSDE", "USDT", "USDC", "USDE"]))
        & (df_lending_borrow.dt >= "2024-06-15")
    ]
    ,
    x="dt",
    y="app_token_tvl_usd",
    color="token",
    title="App Token TVL (USD) Over Time",
    labels={"dt": "Date", "app_token_tvl_usd": "TVL (USD)", "token": "Token"},
)

# Customize layout for better visualization
fig.update_layout(
    template="plotly_white",
    xaxis_title="Date",
    yaxis_title="TVL (USD)",
    legend_title="Token",
    margin=dict(t=50, l=25, r=25, b=50),
)

# Show plot
fig.show()

In [42]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

# Filter data for the pools and lending data
df_pools_filtered = df_pools[
    df_pools["pool"].isin(pools_to_use) &
    (df_pools["timestamp"] >= "2024-06-01") &
    (df_pools["symbol"].isin(["USDE", "SUSDE", "USDT", "USDC"]))
]

df_lending_borrow_filtered = df_lending_borrow[
    (df_lending_borrow["token"] == "SUSDE") &
    (df_lending_borrow["dt"] >= "2024-06-15")
]

# Create the figure for the SUSDE Lending TVL first (dashed black line)
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=df_lending_borrow_filtered["dt"],
        y=df_lending_borrow_filtered["app_token_tvl_usd"],
        mode="lines",
        name="sUSDe AAVE Lending TVL",
        line=dict(color="black", width=2, dash="dash"),
        yaxis="y2"
    )
)

# Add the lines for the lending APY
color_mapping = {
    "SUSDE": "black",         # SUSDE Ethena APY (solid black)
    "USDE": "gray",           # USDE Lending APY (gray)
    "USDT": "green",          # USDT Lending APY (green)
    "USDC": "blue"            # USDC Lending APY (blue)
}

label_mapping = {
    "SUSDE": "sUSDe Ethena Native APY",
    "USDE": "USDe AAVE Lending APY",
    "USDT": "USDT AAVE Lending APY",
    "USDC": "USDC AAVE Lending APY"
}

for symbol in ["SUSDE", "USDE", "USDT", "USDC"]:
    filtered_df = df_pools_filtered[df_pools_filtered["symbol"] == symbol]
    fig.add_trace(
        go.Scatter(
            x=filtered_df["timestamp"],
            y=filtered_df["apy_7d_rolling_avg"],
            mode="lines",
            name=label_mapping[symbol],
            line=dict(color=color_mapping[symbol], width=2)
        )
    )

# Update layout to include a secondary y-axis and adjust the legend position
fig.update_layout(
    template="plotly_white",
    xaxis_title="Date",
    yaxis_title="APY (%)",
    yaxis2=dict(
        title="Lending TVL (USD)",
        overlaying="y",
        side="right",
        showgrid=False
    ),
    legend=dict(
        x=1.05,  # Move the legend slightly to the right
        y=1,     # Align the legend to the top
        xanchor="left", 
        yanchor="top"
    ),
    margin=dict(t=50, l=25, r=50, b=50),
)

# Show plot
fig.show()
